# import packages

In [1]:
# !pip install transformers[torch]==4.3

In [1]:
import os
import torch
import pandas as pd
from transformers import BertModel, DistilBertModel
from tqdm import tqdm
# import kobert_tokenizer

# bring pre-trained kobert model

In [2]:
# pretrained_bertmodel = BertModel.from_pretrained("monologg/kobert")
# 참고
# 아레 코드랑 결과 같음
# model_skt = BertModel.from_pretrained("skt/kobert-base-v1")

In [3]:
os.getcwd()

'C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\1. source code'

# data sets

In [5]:
data_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/1. input data"
file_name = "merged_data_set_AIR.xlsx"
data_set = pd.read_excel(os.path.join(data_path, file_name))
data_set.shape

(48000, 10)

In [6]:
data_set.columns = ['no',
                    'uid',
                   'publisher',
                   'title',
                   'summary',
                   'content',
                   'content_url',
                    'update_at',
                   'importance',
                   'polarity']
data_set = data_set.reset_index()

In [7]:
data_set.head(1)

,index,no,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,0,0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


# Tokenizer

In [7]:
from kobert_transformers import get_tokenizer
tokenizer = get_tokenizer()

In [8]:
tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")

['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']

## For title

In [9]:
data_set_for_title = data_set[['uid', 'title']]
data_set_for_title["token_len"] = ""
data_set_for_title["tokenization"] = ""
# data_set_for_title.reset_index()
data_set_for_title = data_set_for_title.reset_index()

<ipython-input-9-ebfe296a4add>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_title["token_len"] = ""
<ipython-input-9-ebfe296a4add>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_title["tokenization"] = ""


In [72]:
data_set_for_title.head(1)

,index,uid,title,token_len,tokenization
0,0,354652739944452421,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',,


In [73]:
data_set_for_title.tail(1)

,index,uid,title,token_len,tokenization
47999,47999,659762898834428055,"""최신 갤럭시·아이폰, 4명 중 1명이 빌려쓴다""",,


In [74]:
for i in tqdm(range(data_set_for_title.shape[0])):
    tokenized = tokenizer.tokenize(str(data_set_for_title['title'][i]))
    data_set_for_title["token_len"][i] = len(tokenized)
    data_set_for_title["tokenization"][i] = ",".join(tokenized)

  0%|                                                                                        | 0/48000 [00:00<?, ?it/s]<ipython-input-74-9291174cddaf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_title["token_len"][i] = len(tokenized)
<ipython-input-74-9291174cddaf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_title["tokenization"][i] = ",".join(tokenized)
100%|██████████████████████████████████████████████████████████████████████████| 48000/48000 [00:26<00:00, 1833.64it/s]


In [75]:
data_set_for_title.head(2)

,index,uid,title,token_len,tokenization
0,0,354652739944452421,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',22,"▁김진,석,▁CJ,헬,로,비,전,▁대표,▁',주,주,총회,▁70%,이상,▁찬성,으로..."
1,1,848994285528420888,[포토] LG유플러스 AR글라스 국내 최초 출시,13,"▁[,포토,],▁LG,유플러스,▁,AR,글,라,스,▁국내,▁최초,▁출시"


In [76]:
data_set_for_title.tail(2)

,index,uid,title,token_len,tokenization
47998,47998,885647782860427818,귀한 몸 된 갤럭신S20 울트라…유통망 불만 높아진다,20,"▁귀,한,▁몸,▁된,▁,갤,럭,신,S,20,▁울,트,라,...,유통,망,▁불,만,▁..."
47999,47999,659762898834428055,"""최신 갤럭시·아이폰, 4명 중 1명이 빌려쓴다""",17,"▁"",최,신,▁갤럭시,·,아이,폰,,,▁4,명,▁중,▁1,명이,▁빌려,쓴,다,"""


In [67]:
import os
os.getcwd()

'C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\1. source code'

In [77]:
save_path = "C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\2. data sets\\2. processed data"
data_set_for_title.to_excel(save_path + "/tokenization_for_title.xlsx")

## For summary

In [78]:
data_set.head(1)

,index,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,0,354652739944452421,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


In [81]:
data_set_for_summary = data_set[['uid', 'summary']]
data_set_for_summary["token_len"] = ""
data_set_for_summary["tokenization"] = ""
# data_set_for_title.reset_index()
data_set_for_summary = data_set_for_summary.reset_index()

<ipython-input-81-eda7f4fac42b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_summary["token_len"] = ""
<ipython-input-81-eda7f4fac42b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_summary["tokenization"] = ""


In [82]:
data_set_for_summary.head(1)

,index,uid,summary,token_len,tokenization
0,0,354652739944452421,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,,


In [83]:
data_set_for_summary.tail(1)

,index,uid,summary,token_len,tokenization
47999,47999,659762898834428055,SK텔레콤 홍보 모델들이 스마트폰 렌탈 서비스 'T렌탈'을 이용하고 있는 모습.\n...,,


In [91]:
for i in tqdm(range(data_set_for_summary.shape[0])):
    tokenized = tokenizer.tokenize(str(data_set_for_summary['summary'][i]))
    data_set_for_summary["token_len"][i] = len(tokenized)
    data_set_for_summary["tokenization"][i] = ",".join(tokenized)

  0%|                                                                                        | 0/48000 [00:00<?, ?it/s]<ipython-input-91-d3807f5634da>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_summary["token_len"][i] = len(tokenized)
<ipython-input-91-d3807f5634da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_for_summary["tokenization"][i] = ",".join(tokenized)
100%|██████████████████████████████████████████████████████████████████████████| 48000/48000 [00:31<00:00, 1525.82it/s]


In [92]:
data_set_for_summary.head(1)

,index,uid,summary,token_len,tokenization
0,0,354652739944452421,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,90,"▁임,세,영,▁,기자,▁=,▁김진,석,▁CJ,헬,로,비,전,▁대표,가,▁26,일,▁..."


In [93]:
data_set_for_summary.tail(1)

,index,uid,summary,token_len,tokenization
47999,47999,659762898834428055,SK텔레콤 홍보 모델들이 스마트폰 렌탈 서비스 'T렌탈'을 이용하고 있는 모습.\n...,62,"▁SK,텔레콤,▁홍보,▁모델,들이,▁스마트폰,▁,렌,탈,▁서비스,▁',T,렌,탈,'..."


In [94]:
save_path = "C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\2. data sets\\2. processed data"
data_set_for_summary.to_excel(save_path + "/tokenization_for_summary.xlsx")

## For content

In [95]:
data_set_tokened = data_set
data_set_tokened["content_len"] = data_set_tokened['content'].str.len()
data_set_tokened["content_token_len"] = ""
data_set_tokened["content_tokenized"] = ""
data_set_tokened = data_set_tokened.reset_index()

In [96]:
data_set_tokened.head(1)

,level_0,index,no,uid,publisher,title,summary,content,content_url,update_at,importance,polarity,content_len,content_token_len,content_tokenized
0,0,0,0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+,175.0,,


In [97]:
for i in tqdm(range(data_set_tokened.shape[0])):
    tokenized = tokenizer.tokenize(str(data_set_tokened['content'][i]))
    data_set_tokened["content_token_len"][i] = len(tokenized)
    data_set_tokened["content_tokenized"][i] = ",".join(tokenized)

  0%|                                                                                        | 0/48000 [00:00<?, ?it/s]<ipython-input-97-6259c18d9197>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_tokened["content_token_len"][i] = len(tokenized)
<ipython-input-97-6259c18d9197>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_tokened["content_tokenized"][i] = ",".join(tokenized)
100%|███████████████████████████████████████████████████████████████████████████| 48000/48000 [00:53<00:00, 902.99it/s]


In [121]:
data_set_tokened.head(1)

NameError: name 'data_set_tokened' is not defined

In [40]:
data_save_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/2. processed data"
data_set_tokened.to_excel(data_save_path + "/data_set_tokened.xlsx")

# Pre-processing

In [1]:
import os
import torch
import pandas as pd
import re
from transformers import BertModel, DistilBertModel
from tqdm import tqdm
# import kobert_tokenizer

In [2]:
data_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/1. input data"
file_name = "merged_data_set_AIR.xlsx"
data_set = pd.read_excel(os.path.join(data_path, file_name))
data_set.shape

(48000, 10)

In [3]:
data_set.head(1)

,Unnamed: 0,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


In [3]:
data_save_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/2. processed data"
data_set_tokened = pd.read_excel(data_save_path + "/data_set_tokened.xlsx")
data_set_tokened['content'] = data_set_tokened['content'].astype(str)
data_set_tokened.head(1)

,Unnamed: 0,level_0,index,no,uid,publisher,title,summary,content,content_url,update_at,importance,polarity,content_len,content_token_len,content_tokenized
0,0,0,0,0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+,175.0,98,"▁임,세,영,▁,기자,▁=,▁김진,석,▁CJ,헬,로,비,전,▁대표,가,▁26,일,▁..."


In [74]:
# data_set_tokened.publisher.value_counts().to_excel(data_save_path + '/publisher_count.xlsx')

In [55]:
data_set_tokened.loc[9250, 'content_1']

'대기업 최초로 주 35시간 근무제를 도입한 신세계가 계열사 별로 한 달에 하루씩 휴무제를 시행하고 있는데요.  직원들은 본인 의사와 관계없이 강제로 연차휴가를 써야 해서 논란이 되고 있습니다.   가 취재했습니다.    신세계 계열사 직원들은 원하는 날 연차휴가를 쓰기 어렵습니다.  계열사 별로 매달 하루 정해진 날에 한꺼번에 쉬어야 하기 때문입니다.  1년 연차   가운데  을 이렇게 회사가 정해준 날에 써야 합니다.   신세계 계열사 직원    놀러 가고 싶을 때가 있잖아요. 그럴 때  좀 사용을 하고 싶은데 아쉬운 면도 있었고  특히 급한 일이 있을 때 .    일부 계열사는 쉬는 날 연차를 쓰지 않는 직원들에게 다른 계열사 업무를 지원하게 합니다.   신세계 계열사 직원    주로 가면 마트 쪽에서 물건 포장하는  일하는 거 같아요. 연차 안 깎이려고 그렇게 나가서 일하는 거 같습니다.    신세계 측은 자원자에 한해서 계열사 업무지원이 이뤄지고   노사협의회 합의를 통해 일괄 휴가를 실시하고 있다고 말합니다.   최진용 신세계 SSG닷컴 인사팀장    특정일 날 다 같이 쉬는 게 사원들 입장에  서는 실제로 쉰다는 느낌을 가장 잘 받  을 수 있거든요. 주변에서 연락 오는 것도  없고요.    하지만 신세계 계열사 30여 곳 가운데 이마트 등 3곳을 제외하고는 노조가 없어   노사협의회 결정 자체도 논란의 소지가 있다는 지적입니다.    근로기준법에는 근로자가 원하는 날 휴가를 주도록 규정돼 있습니다.   '

In [56]:
data_set_tokened.loc[5750, 'content']

'‘삼성 인사…고동진 부사장, 삼성전자 무선사업부 사장 승진’\n\n삼성그룹은 1일 삼성전자 고동진 부사장을 삼성전자 무선사업부 사장으로 승진해 발표했다고 경향신문이 보도했다.\n\n고동진 사장은 1961년생으로 성균관대 산업공학과를 나와 영국 서섹스대에서 기술정책으로 석사학위를 받았다.\n\n1984년 삼성전자 개발관리과로 입사해 통신연구소 연구운영팀, 인사팀, 인력팀, 정보통신총괄 유럽연구소장, 무선사업부 해외상품기획그룹장, 무선사업부 개발관리팀장, 무선사업부 기술전략팀장을 거쳤다. 고동진 사장은 지난해 12월부터 무선사업부 개발실장을 맡고 있었다.\n\n삼성그룹은 고동진 사장 선임에 대해 “기술기획 업무를 시작으로 정보통신부문의 유럽연구소장을 역임한 후 무선사업부로 자리를 옮겨 상품기획, 기술전략 등 다양한 업무를 두루 경험하며 갤럭시의 성공신화를 이끌어 온 인물”이라며 “특히 2014년말 무선사업부 개발실장으로 부임해 갤럭시 S6, 노트5 등 차별화된 플래그십 모델 개발을 선도했다”고 밝혔다.\n\n삼성그룹은 또한 “고 사장은 H/W 및 S/W는 물론 KNOX, 삼성페이 등 솔루션 및 서비스 개발에도 폭넓은 안목과 식견을 갖추고 있어 무선사업의 제2 도약을 이끌 수 있을 것으로 기대된다”고 밝혔다.\n\n스포츠경향 뉴스를 네이버 뉴스 스탠드에서 만나 보세요. \n모바일 경향 [ |  |  ] | 공식 SNS 계정'

In [44]:
data_set_tokened['content_1'] = data_set_tokened['content']
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('연합뉴스TV.*jebo23', ' ', regex = True)
#data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+=[가-힣]+ 기자', ' ', regex = True)

## Backup

### Exact matching

In [1]:
# data_set_tokened['content_1'] = data_set_tokened['content_1']

## 신문사별
### 1 이데일리
# for row_idx, row in data_set_tokened.iterrows():
#     raw = row['content_1']
#     s = re.search(r'[가-힣]+ [(]+[a-zA-Z0-9.]+[@][a-zA-Z.]+[)]', raw)
#     if s is None:
#         data_set_tokened.loc[row_idx, 'content_1'] = raw
#     else:
#         cleaned = s.string[0:(s.start() - 1)]
#         data_set_tokened.loc[row_idx, 'content_1'] = cleaned
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('이데일리 기자들의 비밀공간', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('이데일리', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("[(]출처: [a-zA-Z0-9가-힣]+[)]", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("단위 : 천원", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("화면번호 : [0-9]+", ' ', regex = True)


### 2 머니투데이
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("이미지/사진제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)


### 3 연합뉴스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('대담=', ' ', regex = False) #2454
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('정리=', ' ', regex = False) #2454
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('사진=', ' ', regex = False) #2454


### 4 한국경제
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한경로보뉴스', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[표]', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[그래프]', ' ', regex = False) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ 한경닷컴 기자', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한경닷컴 [가-힣]+ 기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(현지시간)', ' ', regex = False) # 213 

### 5 아시아경제
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('아시아경제 [가-힣]+ 기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('asiae.co.kr', " ", regex=False)

### 6 뉴시스
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("【[가-힣]+=뉴시스】", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("공감언론 뉴시스통신사", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(현지시각)', ' ', regex = False) # 213 


### 7 서울경제
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[서울경제]', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('서울경제', ' ', regex = False) # 213 


### 8 한국경제TV
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('ⓒ 한국경제TV, 무단 전재 및 재배포 금지', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한국경제TV', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('무단 전재 및 재배포 금지', ' ', regex = False) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('그림 [0-9]+', ' ', regex = True) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('무단 전재 및 재배포 금지', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('라이온봇기자', ' ', regex = False) #213
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('※ 본 기사는 한국경제TV와 `금융 AI 전문기업 씽크풀`이 실시간으로 작성한 기사입니다.', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('동영상 뉴스', ' ', regex = False) #213
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('<기자>', ' ', regex = False) #213
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[인터뷰]', ' ', regex = False) #213

### 9 파이낸셜뉴스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('\'fnRASSI\'는 금융 AI 전문기업 씽크풀과 파이낸셜뉴스의 협업으로 로봇기자가 실시간으로 생산하는 기사입니다.', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('fnRASSI@fnnews.com fnRASSI', ' ', regex = False) # 213 

### 10 매일경제
### 11 헤럴드경제
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("헤럴드경제=[가-힣]+ 기자", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("헤럴드경제 [가-힣]+ 기자", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("헤럴드경제", ' ', regex = False)

### 12 뉴스1
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z가-힣()]+=뉴스1', ' ', regex = True) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('뉴스1', ' ', regex = False) # 213 

### 13 전자신문
# for row_idx, row in data_set_tokened.iterrows():
#     raw = row['content_1']
#     s = re.search(r'\n\n\n\n\n\n\n\n\n\n', raw)
#     if s is None:
#         data_set_tokened.loc[row_idx, 'content_1'] = raw
#     else:
#         cleaned = s.string[0:(s.start() - 1)]
#         data_set_tokened.loc[row_idx, 'content_1'] = cleaned

data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("전자신문과 씽크풀의 증시분석 전문기자 로봇 ET가 쓴 기사입니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("전자신문 관련뉴스해당 언론사에서 선정하며 로 이동해 볼 수 있습니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("개별 기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("언론사는 개별 기사를 2개 이상 섹션으로 중복 분류할 수 있습니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("전자신문 관련뉴스언론사 페이지로 이동합니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("◆ Report statistics", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("◆ Report briefing", ' ', regex = False)
                
        
### 14 조선비즈
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('chosunbiz.com', ' ', regex = False)
                
### 15 국민일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('※ 이 기사는 국민일보와 엠로보가 개발한 증권뉴스 전용 인공지능 로봇 ‘스톡봇’이 금융감독원 전자공시시스템(DART)과 한국거래소(KRX) 데이터를 토대로 작성한 것입니다. 지속적인 업그레이드를 통해 더욱 풍부하고 정확한 내용을 담아 가겠습니다.', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('스톡봇 기자', ' ', regex = False) #213
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공시 전문으로 이동', ' ', regex = False) #213
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+팀 [가-힣]+ [a-zA-Z0-9.]+[@][a-zA-Z.]+', ' ', regex = True)

### 16 데일리안
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('데일리안 스팟뉴스2', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('데일리안 스팟뉴스팀', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('데일리안', ' ', regex = False)

### 17 디지털타임스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('인터넷마케팅팀', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('인터넷 마케팅팀', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('디지털타임스', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('디지털뉴스부', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('문의:', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("핫 섹션", ' ', regex = True)

### 18 머니S
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('@머니S MNB', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("그래프=[a-zA-Z0-9가-힣]+", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진제공", ' ', regex = False)

### 19 동아일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("원본|", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("기획·제작|", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("소비자경제부", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ 인턴', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z0-9가-힣]+ 제공', " ", regex=True)
# for row_idx, row in data_set_tokened.iterrows():
#     raw = row['content_1']
#     s = re.search(r'특별취재팀', raw)
#     if s is None:
#         data_set_tokened.loc[row_idx, 'content_1'] = raw
#     else:
#         cleaned = s.string[0:(s.start() - 1)]
#         data_set_tokened.loc[row_idx, 'content_1'] = cleaned

### 20 중앙일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("중앙일보디자인", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("온라인 중앙일보", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("로컬편집기사 기자", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('글, 사진=', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("[ⓒ 조인스랜드 : DramaHouse & J Content Hub Co.,Ltd.", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z가-힣]+=[a-zA-Z가-힣]+ 특파원', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('문의 [0-9]+=[0-9]+', " ", regex=True)

### 21 아이뉴스24

### 22 부산일보
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진=[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진-[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[→]', " 다음 ", regex=True)

### 23 SBS CNBC
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("자세한 내용은 동영상을 시청하시기 바랍니다.", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("동영상 뉴스", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("경제와이드 이슈&", ' ', regex = False)

### 24 세계일보


### 25 YTN
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('저작권자(c) YTN & YTN PLUS.', " ", regex=False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [\[]+[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+[\]]', " ", regex=True)

### 26 한국일보
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한국일보 [가-힣]+팀', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한국스포츠경제', " ", regex=False)
       
### 27 서울신문
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('온라인뉴스부', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('나우뉴스부', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('나우뉴스', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('재미있는 세상', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('서울비즈', " ", regex=False)

### 28 노컷뉴스
### 29 조선일보
### 30 경향신문
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('경향비즈 SNS', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('모바일 경향', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('모바일경향', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공식 SNS 계정', " ", regex=False)

### 31 문화일보
### 32 스포츠조선
### 33 MBN뉴스
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집 : [가-힣]+', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집: [가-힣]+', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집:[가-힣]+', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집 : [가-힣]+', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재: [가-힣]+', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재:[가-힣]+', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재 : [가-힣]+', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('MBN뉴스 [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('동영상 뉴스', " ", regex=False)


### 34 조세일보
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+[(][a-zA-Z0-9.-]+[@][a-zA-Z.]+[)]', ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ 전문위원', " ", regex = True)

### 35 머니위크
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+=[가-힣]+ 기자', ' ', regex = True) # 광주=이재호 기자

### 36 한겨레
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공식 SNS', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('Weconomy 홈페이지 바로가기:', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('http://www.hani.co.kr/arti/economy', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('Weconomy 페이스북 바로가기:', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('https://www.facebook.com/econohani', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('지금 여기', " ", regex=False)

### 37 스포츠서울
### 38 연합뉴스TV
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('동영상 뉴스', ' ', regex = False)

### 39 스포츠경향
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('스포츠경향 뉴스를 네이버 뉴스 스탠드에서 만나 보세요', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('모바일 경향', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공식 SNS 계정', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('온라인뉴스팀', ' ', regex = False)

# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[↑]', " 상승 ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[→]', " 다음 ", regex=True)


# 공통
## 기자
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 선임기자', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 유통전문기자', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 객원기자', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 인턴기자', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('/[가-힣]+기자', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+기자', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 기자', " ", regex=True)

### 사진 출처
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('자료사진=[a-zA-Z0-9가-힣]+', ' ', regex = True) # 광주=이재호 기자
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("자료=[a-zA-Z0-9가-힣]+", ' ', regex = True) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진=[a-zA-Z0-9가-힣 ]+ 제공", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진=[a-zA-Z0-9가-힣]+", ' ', regex = True)

## e-mail address
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [(]+[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+[)]', ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [a-zA-Z0-9.]+[@][a-zA-Z0-9.]+', ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+[(]+[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+[)]', ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+', ' ', regex = True)

## web site address
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z0-9]+[.][a-zA-Z0-9.]+[.][a-zA-Z0-9.]+', ' ', regex = True)

## stock fs ticker no
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[(][0-9]{6}[)]', ' ', regex = True)
# yyyy.mm.dd
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[0-9]+[.][0-9]+[.][0-9]+', ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[0-9]+일', " ", regex=True)
# 전화번호 (00-000-0000)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[0-9]+[-][0-9]+[-][0-9]+', ' ', regex = True)


## 사진 설명
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가운데)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(오른쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(사진 오른쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가장 오른쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(왼쪽)', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(왼쪽 [가-힣]+번째)', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가장 왼쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(왼쪽부터)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[표]', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[그래프]', ' ', regex = False) # 213 

## 기타 특수문자 처리
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[()·]', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[【】\[\]▽△▲▲■◇♦◆○●ⓒΔ▷▶�━ㆍ㈜]', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('\n', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('\u3000', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace(u'\xa0', u" ", regex=True)

# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[-=+,#/\/\?:^$@*\"※~&%ㆍ!』\\‘|\[\]\<\>`\'…’》│]', " ", regex=True)

NameError: name 'data_set_tokened' is not defined

### Regular expression

In [ ]:
## 신문사별
### 1 이데일리
for row_idx, row in data_set_tokened.iterrows():
    raw = row['content_1']
    s = re.search(r'[가-힣]+ [(]+[a-zA-Z0-9.]+[@][a-zA-Z.]+[)]', raw)
    if s is None:
        data_set_tokened.loc[row_idx, 'content_1'] = raw
    else:
        cleaned = s.string[0:(s.start() - 1)]
        data_set_tokened.loc[row_idx, 'content_1'] = cleaned
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('이데일리 기자들의 비밀공간', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('이데일리', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("[(]출처: [a-zA-Z0-9가-힣]+[)]", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("단위 : 천원", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("화면번호 : [0-9]+", ' ', regex = True)


### 2 머니투데이
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("이미지/사진제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)


### 3 연합뉴스
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('대담=', ' ', regex = False) #2454
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('정리=', ' ', regex = False) #2454
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('사진=', ' ', regex = False) #2454


### 4 한국경제
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한경로보뉴스', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[표]', ' ', regex = False) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[그래프]', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ 한경닷컴 기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한경닷컴 [가-힣]+ 기자', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(현지시간)', ' ', regex = False) # 213 

### 5 아시아경제
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('아시아경제 [가-힣]+ 기자', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('asiae.co.kr', " ", regex=False)

### 6 뉴시스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("【[가-힣]+=뉴시스】", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("공감언론 뉴시스통신사", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(현지시각)', ' ', regex = False) # 213 


### 7 서울경제
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[서울경제]', ' ', regex = False) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('서울경제', ' ', regex = False) # 213 


### 8 한국경제TV
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('ⓒ 한국경제TV, 무단 전재 및 재배포 금지', ' ', regex = False) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한국경제TV', ' ', regex = False) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('무단 전재 및 재배포 금지', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('그림 [0-9]+', ' ', regex = True) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('무단 전재 및 재배포 금지', ' ', regex = False) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('라이온봇기자', ' ', regex = False) #213
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('※ 본 기사는 한국경제TV와 `금융 AI 전문기업 씽크풀`이 실시간으로 작성한 기사입니다.', ' ', regex = False) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('동영상 뉴스', ' ', regex = False) #213
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('<기자>', ' ', regex = False) #213
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[인터뷰]', ' ', regex = False) #213

### 9 파이낸셜뉴스
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('\'fnRASSI\'는 금융 AI 전문기업 씽크풀과 파이낸셜뉴스의 협업으로 로봇기자가 실시간으로 생산하는 기사입니다.', ' ', regex = False) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('fnRASSI@fnnews.com fnRASSI', ' ', regex = False) # 213 

### 10 매일경제
### 11 헤럴드경제
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("헤럴드경제=[가-힣]+ 기자", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("헤럴드경제 [가-힣]+ 기자", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("헤럴드경제", ' ', regex = False)

### 12 뉴스1
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z가-힣()]+=뉴스1', ' ', regex = True) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('뉴스1', ' ', regex = False) # 213 

### 13 전자신문
for row_idx, row in data_set_tokened.iterrows():
    raw = row['content_1']
    s = re.search(r'\n\n\n\n\n\n\n\n\n\n', raw)
    if s is None:
        data_set_tokened.loc[row_idx, 'content_1'] = raw
    else:
        cleaned = s.string[0:(s.start() - 1)]
        data_set_tokened.loc[row_idx, 'content_1'] = cleaned

# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("전자신문과 씽크풀의 증시분석 전문기자 로봇 ET가 쓴 기사입니다", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("전자신문 관련뉴스해당 언론사에서 선정하며 로 이동해 볼 수 있습니다", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("개별 기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("언론사는 개별 기사를 2개 이상 섹션으로 중복 분류할 수 있습니다", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("전자신문 관련뉴스언론사 페이지로 이동합니다", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("◆ Report statistics", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("◆ Report briefing", ' ', regex = False)
                
        
### 14 조선비즈
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('chosunbiz.com', ' ', regex = False)
                
### 15 국민일보
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('※ 이 기사는 국민일보와 엠로보가 개발한 증권뉴스 전용 인공지능 로봇 ‘스톡봇’이 금융감독원 전자공시시스템(DART)과 한국거래소(KRX) 데이터를 토대로 작성한 것입니다. 지속적인 업그레이드를 통해 더욱 풍부하고 정확한 내용을 담아 가겠습니다.', ' ', regex = False) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('스톡봇 기자', ' ', regex = False) #213
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공시 전문으로 이동', ' ', regex = False) #213
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+팀 [가-힣]+ [a-zA-Z0-9.]+[@][a-zA-Z.]+', ' ', regex = True)

### 16 데일리안
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('데일리안 스팟뉴스2', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('데일리안 스팟뉴스팀', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('데일리안', ' ', regex = False)

### 17 디지털타임스
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('인터넷마케팅팀', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('인터넷 마케팅팀', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('디지털타임스', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('디지털뉴스부', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('문의:', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("핫 섹션", ' ', regex = True)

### 18 머니S
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('@머니S MNB', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("그래프=[a-zA-Z0-9가-힣]+", ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진제공", ' ', regex = False)

### 19 동아일보
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("원본|", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("기획·제작|", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("소비자경제부", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ 인턴', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z0-9가-힣]+ 제공', " ", regex=True)
for row_idx, row in data_set_tokened.iterrows():
    raw = row['content_1']
    s = re.search(r'특별취재팀', raw)
    if s is None:
        data_set_tokened.loc[row_idx, 'content_1'] = raw
    else:
        cleaned = s.string[0:(s.start() - 1)]
        data_set_tokened.loc[row_idx, 'content_1'] = cleaned

### 20 중앙일보
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("중앙일보디자인", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("온라인 중앙일보", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("로컬편집기사 기자", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('글, 사진=', " ", regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("[ⓒ 조인스랜드 : DramaHouse & J Content Hub Co.,Ltd.", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z가-힣]+=[a-zA-Z가-힣]+ 특파원', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('문의 [0-9]+=[0-9]+', " ", regex=True)

### 21 아이뉴스24

### 22 부산일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진=[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진-[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[→]', " 다음 ", regex=True)

### 23 SBS CNBC
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("자세한 내용은 동영상을 시청하시기 바랍니다.", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("동영상 뉴스", ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("경제와이드 이슈&", ' ', regex = False)

### 24 세계일보


### 25 YTN
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('저작권자(c) YTN & YTN PLUS.', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [\[]+[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+[\]]', " ", regex=True)

### 26 한국일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한국일보 [가-힣]+팀', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한국스포츠경제', " ", regex=False)
       
### 27 서울신문
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('온라인뉴스부', " ", regex=False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('나우뉴스부', " ", regex=False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('나우뉴스', " ", regex=False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('재미있는 세상', " ", regex=False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('서울비즈', " ", regex=False)

### 28 노컷뉴스
### 29 조선일보
### 30 경향신문
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('경향비즈 SNS', " ", regex=False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('모바일 경향', " ", regex=False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('모바일경향', " ", regex=False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공식 SNS 계정', " ", regex=False)

### 31 문화일보
### 32 스포츠조선
### 33 MBN뉴스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집 : [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집: [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집:[가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집 : [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재: [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재:[가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재 : [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('MBN뉴스 [가-힣]+', " ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집', " ", regex=False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재', " ", regex=False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('동영상 뉴스', " ", regex=False)


### 34 조세일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+[(][a-zA-Z0-9.-]+[@][a-zA-Z.]+[)]', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ 전문위원', " ", regex = True)

### 35 머니위크
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+=[가-힣]+ 기자', ' ', regex = True) # 광주=이재호 기자


### 36 한겨레
### 37 스포츠서울
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('제공 [\|] [가-힣]+', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('스포츠서울 [가-힣]+기자', " ", regex = True)


### 38 연합뉴스TV
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('연합뉴스TV.*jebo23', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('연합뉴스TV.*yjebo@yna.co.kr', ' ', regex = True)

### 38 KBS 뉴스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('KBS 뉴스.*입니다.', ' ', regex = True)

### 39 스포츠경향
###
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[↑]', " 상승 ", regex=True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[→]', " 다음 ", regex=True)


# 공통
## 기자
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 선임기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 유통전문기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 객원기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 인턴기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('/[가-힣]+기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 기자', " ", regex=True)

### 사진 출처
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('자료사진=[a-zA-Z0-9가-힣]+', ' ', regex = True) # 광주=이재호 기자
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("자료=[a-zA-Z0-9가-힣]+", ' ', regex = True) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진=[a-zA-Z0-9가-힣 ]+ 제공", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진=[a-zA-Z0-9가-힣]+", ' ', regex = True)

## e-mail address
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [(]+[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+[)]', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [a-zA-Z0-9.]+[@][a-zA-Z0-9.]+', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+[(]+[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+[)]', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+', ' ', regex = True)

## web site address
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z0-9]+[.][a-zA-Z0-9.]+[.][a-zA-Z0-9.]+', ' ', regex = True)

## stock fs ticker no
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[(][0-9]{6}[)]', ' ', regex = True)
# yyyy.mm.dd
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[0-9]+[.][0-9]+[.][0-9]+', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[0-9]+일', " ", regex=True)
# 전화번호 (00-000-0000)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[0-9]+[-][0-9]+[-][0-9]+', ' ', regex = True)


## 사진 설명
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가운데)', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(오른쪽)', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(사진 오른쪽)', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가장 오른쪽)', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(왼쪽)', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(왼쪽 [가-힣]+번째)', ' ', regex = True)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가장 왼쪽)', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(왼쪽부터)', ' ', regex = False)
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[표]', ' ', regex = False) # 213 
# data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[그래프]', ' ', regex = False) # 213 

## 기타 특수문자 처리
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[()·]', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[【】\[\]▽△▲▲■◇♦◆○●◎ⓒΔ▷▶�━ㆍ㈜]', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('\n', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('\u3000', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace(u'\xa0', u" ", regex=True)

data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[-=+,#/\/\?:^$@*\"※~&%ㆍ!』\\‘|\[\]\<\>`\'…’》│“”]', " ", regex=True)

In [9]:
data_set_tokened.loc[9987, 'content']

"■ 방송 : YTN 라디오 FM 94.5 (15:10~16:00)\n■ 진행 : 김혜민 PD\n■ 대담 : 백상희 특허법인 테헤란 변리사\n\n* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다.\n\n[생생경제] 신세계, BTS 위해 'BTS(Boon The Shop)' 포기해\n\n◇ 김혜민 PD(이하 김혜민)> 오늘 가장 뜨거운 경제뉴스를 제일 생생하게 전해드리는 시간입니다. 상표브로커라고 들어보셨어요? 상표가 등록이 안 돼 있는 것들을 먼저 등록해서 경제적인 이득을 취하려고 하는 건데요. 그래서 펭수도 EBS가 아닌 제 3자가 등록을 하고, 또 한 기업에서 BTS로 상표권을 출연하려고 해서 논란이 되기도 했었습니다. 개인 방송 채널이나 개인 블로그, SNS 등이 활발하게 되면서 이런 문제가 더 빈번하게 벌어지는 것 같은데요. 관련 내용 특허법인 테헤란 백상희 변리사 전화연결해서 들어보겠습니다. 변리사님, 안녕하세요?\n\n◆ 백상희 특허법인 테헤란 변리사(이하 백상희)> 네, 안녕하세요. 백상희 변리사입니다.\n\n◇ 김혜민> 네, 반갑습니다. 제가 방금 이야기한 것처럼 최근 들어 이런 개인 방송 채널, 블로그, SNS 등이 활발해지면서 특허권, 상품권 논란이 더 많아지는 것 같은데, 맞습니까?\n\n◆ 백상희> 네, 맞습니다. 이런 분쟁은 예전부터 계속 있기는 했는데요. 최근에 특히 유튜브 방송이 활발하게 사람들이 이용하면서 이런 채널명이라든지, 상표명이라든지, 이런 것들에 대한 분쟁이 최근에 상당히 늘어난 경향이 있습니다.\n\n◇ 김혜민> 그러니까 예전에는 기업 대 기업이라든지, 사업하시는 분들 사이에서만 일어났던 분쟁들이 이제는 개인들이 브랜드를 너무 많이 갖게 되는 세상이 오면서 분쟁이 훨씬 더 다양해지고 많아진 거죠?\n\n◆ 백상희> 네, 맞습니다.\n\n◇ 김혜민> 본격적으로 논란을 짚어보기 전에 변리사님께서 상표권이라는 개념에 대해서 쉽게 정리를 먼저 해주시면 좋을 것 같습니다.\

## 신문사별 특징문구(long)

In [4]:
data_set_tokened['content_1'] = data_set_tokened['content']

### Exact matching

In [5]:
## 신문사별
### 1 이데일리
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('이데일리 기자들의 비밀공간', ' ', regex = False)
### 2 머니투데이
### 3 연합뉴스
### 4 한국경제
### 5 아시아경제
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('asiae.co.kr', " ", regex = False)

### 6 뉴시스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("공감언론 뉴시스통신사", ' ', regex = False)

### 7 서울경제
### 8 한국경제TV
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('ⓒ 한국경제TV, 무단 전재 및 재배포 금지', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('무단 전재 및 재배포 금지', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('※ 본 기사는 한국경제TV와 `금융 AI 전문기업 씽크풀`이 실시간으로 작성한 기사입니다.', ' ', regex = False) # 213 

### 9 파이낸셜뉴스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('\'fnRASSI\'는 금융 AI 전문기업 씽크풀과 파이낸셜뉴스의 협업으로 로봇기자가 실시간으로 생산하는 기사입니다.', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('fnRASSI@fnnews.com fnRASSI', ' ', regex = False) # 213 

### 10 매일경제
### 11 헤럴드경제
### 12 뉴스1
### 13 전자신문
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("전자신문과 씽크풀의 증시분석 전문기자 로봇 ET가 쓴 기사입니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("전자신문 관련뉴스해당 언론사에서 선정하며 로 이동해 볼 수 있습니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("개별 기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("언론사는 개별 기사를 2개 이상 섹션으로 중복 분류할 수 있습니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("전자신문 관련뉴스언론사 페이지로 이동합니다", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("◆ Report statistics", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("◆ Report briefing", ' ', regex = False)
                
### 14 조선비즈
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('chosunbiz.com', ' ', regex = False)
                
### 15 국민일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('※ 이 기사는 국민일보와 엠로보가 개발한 증권뉴스 전용 인공지능 로봇 ‘스톡봇’이 금융감독원 전자공시시스템(DART)과 한국거래소(KRX) 데이터를 토대로 작성한 것입니다. 지속적인 업그레이드를 통해 더욱 풍부하고 정확한 내용을 담아 가겠습니다.', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공시 전문으로 이동', ' ', regex = False) #213

### 16 데일리안
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('데일리안 스팟뉴스2', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('데일리안 스팟뉴스팀', ' ', regex = False)

### 17 디지털타임스
### 18 머니S
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('@머니S MNB', ' ', regex = False)

### 19 동아일보
### 20 중앙일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("[ⓒ 조인스랜드 : DramaHouse & J Content Hub Co.,Ltd.", ' ', regex = False)

### 21 아이뉴스24
### 22 부산일보
### 23 SBS CNBC
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("자세한 내용은 동영상을 시청하시기 바랍니다.", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("경제와이드 이슈&", ' ', regex = False)

### 24 세계일보
### 25 YTN
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('저작권자(c) YTN & YTN PLUS.', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다.', " ", regex = False)

### 26 한국일보
### 27 서울신문
### 28 노컷뉴스
### 29 조선일보
### 30 경향신문
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('경향비즈 SNS', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공식 SNS 계정', " ", regex=False)

### 31 문화일보
### 32 스포츠조선
### 33 MBN뉴스
### 34 조세일보
### 35 머니위크
### 36 한겨레
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('Weconomy 홈페이지 바로가기:', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('http://www.hani.co.kr/arti/economy', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('Weconomy 페이스북 바로가기:', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('https://www.facebook.com/econohani', " ", regex = False)

### 37 스포츠서울
### 38 연합뉴스TV
### 39 스포츠경향
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('스포츠경향 뉴스를 네이버 뉴스 스탠드에서 만나 보세요', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공식 SNS 계정', ' ', regex = False)


### Regular expression

In [6]:
## 신문사별
### 1 이데일리
data_set_tokened_tmp = data_set_tokened[data_set_tokened["publisher"] == "이데일리"]
for row_idx, row in data_set_tokened_tmp.iterrows():
    raw = row['content_1']
    s = re.search(r'[가-힣]+ [(]+[a-zA-Z0-9.]+[@][a-zA-Z.]+[)]', raw)
    if s is None:
        data_set_tokened.loc[row_idx, 'content_1'] = raw
    else:
        cleaned = s.string[0:(s.start() - 1)]
        data_set_tokened.loc[row_idx, 'content_1'] = cleaned
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("[(]출처: [a-zA-Z0-9가-힣]+[)]", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("화면번호 : [0-9]+", ' ', regex = True)

### 2 머니투데이
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("이미지/사진제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)

### 3 연합뉴스
### 4 한국경제
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ 한경닷컴 기자', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한경닷컴 [가-힣]+ 기자', " ", regex = True)

### 5 아시아경제
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('아시아경제 [가-힣]+ 기자', " ", regex = True)

### 6 뉴시스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("【[가-힣]+=뉴시스】", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("공감언론 뉴시스통신사", ' ', regex = True)

### 7 서울경제

### 8 한국경제TV
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('그림 [0-9]+', ' ', regex = True)

### 9 파이낸셜뉴스
### 10 매일경제
### 11 헤럴드경제
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("헤럴드경제=[가-힣]+ 기자", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("헤럴드경제 [가-힣]+ 기자", ' ', regex = True)

### 12 뉴스1
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z가-힣()]+=뉴스1', ' ', regex = True)

### 13 전자신문
data_set_tokened_tmp = data_set_tokened[data_set_tokened["publisher"] == "전자신문"]
for row_idx, row in data_set_tokened_tmp.iterrows():
    raw = row['content_1']
    s = re.search(r'\n\n\n\n\n\n\n\n\n\n', raw)
    if s is None:
        data_set_tokened.loc[row_idx, 'content_1'] = raw
    else:
        cleaned = s.string[0:(s.start() - 1)]
        data_set_tokened.loc[row_idx, 'content_1'] = cleaned
        
### 14 조선비즈
### 15 국민일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+팀 [가-힣]+ [a-zA-Z0-9.]+[@][a-zA-Z.]+', ' ', regex = True)

### 16 데일리안
### 17 디지털타임스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)

### 18 머니S
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("그래프=[a-zA-Z0-9가-힣]+", ' ', regex = True)

### 19 동아일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ 인턴', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z0-9가-힣]+ 제공', " ", regex = True)

data_set_tokened_tmp = data_set_tokened[data_set_tokened["publisher"] == "동아일보"]
for row_idx, row in data_set_tokened.iterrows():
    raw = row['content_1']
    s = re.search(r'특별취재팀', raw)
    if s is None:
        data_set_tokened.loc[row_idx, 'content_1'] = raw
    else:
        cleaned = s.string[0:(s.start() - 1)]
        data_set_tokened.loc[row_idx, 'content_1'] = cleaned

### 20 중앙일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z가-힣]+=[a-zA-Z가-힣]+ 특파원', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('문의 [0-9]+=[0-9]+', " ", regex = True)

### 21 아이뉴스24
### 22 부산일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진=[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진-[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[→]', " 다음 ", regex = True)

### 23 SBS CNBC
### 24 세계일보
### 25 YTN
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [\[]+[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+[\]]', " ", regex = True)

### 26 한국일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한국일보 [가-힣]+팀', " ", regex = True)
       
### 27 서울신문
### 28 노컷뉴스
### 29 조선일보
### 30 경향신문
### 31 문화일보
### 32 스포츠조선
### 33 MBN뉴스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집 : [가-힣]+', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집: [가-힣]+', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집:[가-힣]+', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집 : [가-힣]+', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재: [가-힣]+', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재:[가-힣]+', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재 : [가-힣]+', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('MBN뉴스 [가-힣]+', " ", regex = True)

### 34 조세일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+[(][a-zA-Z0-9.-]+[@][a-zA-Z.]+[)]', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ 전문위원', " ", regex = True)

### 35 머니위크
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+=[가-힣]+ 기자', ' ', regex = True) # 광주=이재호 기자

### 36 한겨레
### 37 스포츠서울
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('제공 [\|] [가-힣]+', " ", regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('스포츠서울 [가-힣]+기자', " ", regex = True)

### 38 연합뉴스TV
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('연합뉴스TV.*jebo23', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('연합뉴스TV.*yjebo@yna.co.kr', ' ', regex = True)

### 38 KBS 뉴스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('KBS 뉴스.*입니다.', ' ', regex = True)

### 39 스포츠경향

## 신문사별 특징문구(short)

### Exact mathchimg

In [7]:
## 신문사별
### 1 이데일리
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('이데일리', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("단위 : 천원", ' ', regex = False)

### 2 머니투데이
### 3 연합뉴스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('대담=', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('정리=', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('사진=', ' ', regex = False) 

### 4 한국경제
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한경로보뉴스', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[표]', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[그래프]', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(현지시간)', ' ', regex = False)

### 5 아시아경제
### 6 뉴시스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(현지시각)', ' ', regex = False)

### 7 서울경제
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[서울경제]', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('서울경제', ' ', regex = False)


### 8 한국경제TV
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한국경제TV', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('라이온봇기자', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('동영상 뉴스', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('<기자>', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[인터뷰]', ' ', regex = False)

### 9 파이낸셜뉴스
### 10 매일경제
### 11 헤럴드경제
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("헤럴드경제", ' ', regex = False)

### 12 뉴스1
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('뉴스1', ' ', regex = False)

### 13 전자신문
### 14 조선비즈
### 15 국민일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('스톡봇 기자', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공시 전문으로 이동', ' ', regex = False)
### 16 데일리안
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('데일리안 스팟뉴스2', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('데일리안 스팟뉴스팀', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('데일리안', ' ', regex = False)

### 17 디지털타임스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('인터넷마케팅팀', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('인터넷 마케팅팀', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('디지털타임스', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('디지털뉴스부', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('문의:', ' ', regex = False)

### 18 머니S
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진제공", ' ', regex = False)

### 19 동아일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("원본|", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("기획·제작|", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("소비자경제부", ' ', regex = False)

### 20 중앙일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("중앙일보디자인", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("온라인 중앙일보", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("로컬편집기사 기자", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('글, 사진=', " ", regex = False)

### 21 아이뉴스24
### 22 부산일보
### 23 SBS CNBC
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("동영상 뉴스", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("경제와이드 이슈&", ' ', regex = False)

### 24 세계일보
### 25 YTN
### 26 한국일보
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('한국스포츠경제', " ", regex = False)
       
### 27 서울신문
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('온라인뉴스부', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('나우뉴스부', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('나우뉴스', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('재미있는 세상', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('서울비즈', " ", regex = False)

### 28 노컷뉴스
### 29 조선일보
### 30 경향신문
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('경향비즈 SNS', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('모바일 경향', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('모바일경향', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공식 SNS 계정', " ", regex = False)

### 31 문화일보
### 32 스포츠조선
### 33 MBN뉴스
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('동영상 뉴스', " ", regex = False)


### 34 조세일보
### 35 머니위크
### 36 한겨레
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공식 SNS', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('지금 여기', " ", regex = False)

### 37 스포츠서울
### 38 연합뉴스TV
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('동영상 뉴스', ' ', regex = False)

### 39 스포츠경향
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('모바일 경향', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공식 SNS 계정', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('온라인뉴스팀', ' ', regex = False)

## 공통

In [8]:
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("단위 : 천원", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("화면번호 : [0-9]+", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("이미지/사진제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('대담=', ' ', regex = False) #2454
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('정리=', ' ', regex = False) #2454
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('사진=', ' ', regex = False) #2454
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[표]', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[그래프]', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(현지시간)', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(현지시각)', ' ', regex = False) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('동영상 뉴스', ' ', regex = False) #213
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('<기자>', ' ', regex = False) #213
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[인터뷰]', ' ', regex = False) #213
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('공시 전문으로 이동', ' ', regex = False) #213
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('문의:', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("그래프=[a-zA-Z0-9가-힣]+", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진제공", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("원본|", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("기획·제작|", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('글, 사진=', " ", regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z가-힣]+=[a-zA-Z가-힣]+ 특파원', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('문의 [0-9]+=[0-9]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진=[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진-[a-zA-Z0-9가-힣]+ 제공", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[→]', " 다음 ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("동영상 뉴스", ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [\[]+[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+[\]]', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집 : [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집: [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집:[가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집 : [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재: [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재:[가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재 : [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('MBN뉴스 [가-힣]+', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상편집', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('영상취재', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('동영상 뉴스', " ", regex=False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[↑]', " 상승 ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[→]', " 다음 ", regex=True)

## 기자
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 선임기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 유통전문기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 객원기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 인턴기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('/[가-힣]+기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+기자', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣·]+ 기자', " ", regex=True)

### 사진 및 자료 출처
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('자료사진=[a-zA-Z0-9가-힣]+', ' ', regex = True) # 광주=이재호 기자
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("제공=[a-zA-Z0-9가-힣]+", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("자료=[a-zA-Z0-9가-힣]+", ' ', regex = True) # 213 
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진=[a-zA-Z0-9가-힣 ]+ 제공", ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace("사진=[a-zA-Z0-9가-힣]+", ' ', regex = True)

## e-mail address
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [(]+[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+[)]', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+ [a-zA-Z0-9.]+[@][a-zA-Z0-9.]+', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[가-힣]+[(]+[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+[)]', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z0-9.]+[@][a-zA-Z0-9.]+', ' ', regex = True)

## web site address
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[a-zA-Z0-9]+[.][a-zA-Z0-9.]+[.][a-zA-Z0-9.]+', ' ', regex = True)

## stock fs ticker no
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[(][0-9]{6}[)]', '', regex = True)

# yyyy.mm.dd
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[0-9]+[.][0-9]+[.][0-9]+', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[0-9]+일', " ", regex=True)
# 전화번호 (00-000-0000)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[0-9]+[-][0-9]+[-][0-9]+', ' ', regex = True)

## 사진 설명
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가운데)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(오른쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(사진 오른쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가장 오른쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(왼쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(왼쪽 [가-힣]+번째)', ' ', regex = True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(가장 왼쪽)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('(왼쪽부터)', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[표]', ' ', regex = False)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[그래프]', ' ', regex = False)

# 기타 특수문자 처리
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[()·]', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[【】\[\]▽△▲▲■◇♦◆○●ⓒΔ▷▶�━ㆍ㈜]', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('\n', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('\u3000', " ", regex=True)
data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace(u'\xa0', u" ", regex=True)

data_set_tokened['content_1'] = data_set_tokened['content_1'].str.replace('[-=+,#/\/\?:^$@*\"※~&%ㆍ!』\\‘|\[\]\<\>`\'…’》│]', " ", regex=True)

In [11]:
data_cleaned = data_set_tokened.loc[:, ['uid', 'publisher', 'title', 'summary', 'content', 'content_1', 'content_url', 'update_at',
                                       'importance', 'polarity',
                                       'content_len']]
data_cleaned["content_1_len"] = data_cleaned['content_1'].str.len()

In [12]:
data_cleaned.head(1)

,uid,publisher,title,summary,content,content_1,content_url,update_at,importance,polarity,content_len,content_1_len
0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,김진석 CJ헬로비전 대표가 오전 서울 마포구 상암동 누리꿈스퀘어에서 열린...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+,175.0,158


In [13]:
data_cleaned = data_set_tokened.loc[:, ['uid', 'publisher', 'title', 'summary', 'content', 'content_1', 'content_url', 'update_at',
                                       'importance', 'polarity',
                                       'content_len']]
data_cleaned["content_1_len"] = data_cleaned['content_1'].str.len()

In [14]:
data_cleaned.head(1)

,uid,publisher,title,summary,content,content_1,content_url,update_at,importance,polarity,content_len,content_1_len
0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,김진석 CJ헬로비전 대표가 오전 서울 마포구 상암동 누리꿈스퀘어에서 열린...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+,175.0,158


In [85]:
data_cleaned_test = myRound(data_cleaned, 'content_1_len', 'rd_content_1_len')

In [86]:
data_cleaned_test.head(10)

,uid,publisher,title,summary,content,content_1,content_url,update_at,importance,polarity,content_len,content_1_len,digits,rd_content_1_len
0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,김진석 CJ헬로비전 대표가 오전 서울 마포구 상암동 누리꿈스퀘어에서 열린...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+,175.0,158,3,100
1,848994285528420864,세계일보,[포토] LG유플러스 AR글라스 국내 최초 출시,LG유플러스는 21일 종각직영점에서 AR글라스를 국내 최초로 선보였다.\n\nAR ...,LG유플러스는 21일 종각직영점에서 AR글라스를 국내 최초로 선보였다. AR 글라스...,LG유플러스는 종각직영점에서 AR글라스를 국내 최초로 선보였다. AR 글라스는 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-11-21T13:16:00,0,+,143.0,134,3,100
2,384800874481980224,매일경제,"LG유플러스, 전국 대리점주 200여명과 상생협력 ‘약속’",LG유플러스는 18일부터 19일 경기 광주시 곤지암리조트에 전국 대리점주를 초청해 ...,LG유플러스는 18일부터 19일 경기 광주시 곤지암리조트에 전국 대리점주를 초청해 ...,LG유플러스는 부터 경기 광주시 곤지암리조트에 전국 대리점주를 초청해 상생협력...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-05-19T15:02:00,-,0,367.0,363,3,300
3,323299472392196224,머니투데이,"[사진]LG유플러스, '갤럭시 노트-S6 엣지 플러스 판매'",LG유플러스가 20일 오전 서울 중구 세종대로 시청역 직영점에서 판매를 시작한 '갤...,LG유플러스가 20일 오전 서울 중구 세종대로 시청역 직영점에서 판매를 시작한 '갤...,LG유플러스가 오전 서울 중구 세종대로 시청역 직영점에서 판매를 시작한 갤럭시...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2015-08-20T12:57:00,-,0,83.0,81,2,80
4,809184698633949440,뉴시스,평창 발왕산 트레일 18K 안 뛰면 후회,김경목 기자 = 뉴시스 강원과 용평리조트가 주최하는 '2019 평창 발왕산 트레일 ...,김경목 기자 = 뉴시스 강원과 용평리조트가 주최하는 '2019 평창 발왕산 트레일 ...,뉴시스 강원과 용평리조트가 주최하는 2019 평창 발왕산 트레일 18K ...,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-08-03T16:52:00,-,0,386.0,349,3,300
5,412971802319524224,전자신문,"[@공시]4일 공시종합 - 대표이사변경,최대주주변경(총 2건)",종목명\n(종목코드)\n\n\n변경사유 우리이티아이\n(082850) 윤철주 윤...,종목명\n(종목코드)\n\n\n변경사유 우리이티아이\n(082850) 윤철주 윤...,종목명 종목코드 변경사유 우리이티아이 윤철주 윤철주 이영득 각자 대표...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-08-05T08:42:00,0,0,117.0,101,3,100
6,323478288028798976,뉴시스,"[장 마감후 공시]호텔신라, 3분기 잠정 영업익 전년 比 95.1%↓ 外","발행가 1000원, 상장 예정일 11월16일.\n\n발행가 3620원, 상장 예정일...","▲호텔신라, 연결 기준 2015 회계연도 잠정 3분기 영업이익 28억3300만원으로...",호텔신라 연결 기준 2015 회계연도 잠정 3분기 영업이익 28억3300만원으로...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2015-11-02T07:40:00,0,0,3581.0,3575,4,3000
7,323061449691893824,연합뉴스,"KT, 우즈베키스탄 해외 원격검침 인프라 구축 최종 계약",KT는 우즈베키스탄 전력청의 지능형원격검침(AEM) 구축 사업을 따냈다고 11일 밝...,KT는 우즈베키스탄 전력청의 지능형원격검침(AEM) 구축 사업을 따냈다고 11일 밝...,KT는 우즈베키스탄 전력청의 지능형원격검침 AEM 구축 사업을 따냈다고 밝혔다...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2015-05-11T10:33:00,+,+,454.0,444,3,400
8,323060095875420224,이데일리,"[포토]11번가, 인기 반려동물 용품 최대 60% 할인 판매",SK플래닛 11번가가 10일 오전 서울 중구 을지로 SK텔레콤 T타워에서 인기 반려...,SK플래닛 11번가가 10일 오전 서울 중구 을지로 SK텔레콤 T타워에서 인기 반려...,SK플래닛 11번가가 오전 서울 중구 을지로 SK텔레콤 T타워에서 인기 반려동물...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2015-05-10T11:53:00,-,0,240.0,204,3,200
9,387012294535680896,매일경제,"“메뉴판에 없는 요리”…SKT, 최저 LTE 요금제 숨기기",SK텔레콤이 최저 LTE 요금제 안내를 소홀히 하고 있어 소비자들의 선택권을 제한하...,SK텔레콤이 최저 LTE 요금제 안내를 소홀히 하고 있어 소비자들의 선택권을 제한하...,SK텔레콤이 최저 LTE 요금제 안내를 소홀히 하고 있어 소비자들의 선택권을 제한하...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-05-25T17:29:00,-,0,938.0,922,3,900


In [84]:
def myRound(df, col_name, new_col_name):
    df['digits'] = df[col_name].astype(str).str.len()
    # df['digits'] = df[col_name].str.len()
    df[new_col_name] = df[col_name]
    for i in [5, 4, 3, 2]:
        num_digits = 1
        num_zeros = i - 1 # max(1, i - 2)
        df_tmp = df[df['digits'] == i]
        for idx, row in df_tmp.iterrows():
            raw_col = row[col_name]
            # df.loc[idx, new_col_name] = raw_col.astype(str)[:(i - 3)] + ("0" * (i - 2))
            df.loc[idx, new_col_name] = str(raw_col)[:num_digits] + ("0" * num_zeros)
    return df

In [93]:
content_length_summary = data_cleaned_test['rd_content_1_len'].value_counts()
content_length_summary.to_excel(data_save_path + "/content_length_summary.xlsx")

In [31]:
# data_save_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/2. processed data"
# data_cleaned.to_excel(data_save_path + "/data_cleaned.xlsx")

# Modeling

## Model1 : No preprocessing + KoBert

In [5]:
# packages
import os
import torch
import pandas as pd
from transformers import BertModel, DistilBertModel
from tqdm import tqdm
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

In [6]:
# import data
data_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/1. input data"
file_name = "merged_data_set_AIR.xlsx"
data_set = pd.read_excel(os.path.join(data_path, file_name))
data_set.columns = ['no',
                    'uid',
                   'publisher',
                   'title',
                   'summary',
                   'content',
                   'content_url',
                    'update_at',
                   'importance',
                   'polarity']
data_set = data_set.reset_index()
data_set['content'] = data_set['content'].astype(str)
data_set.head(1)

,index,no,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,0,0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


In [7]:
data_set = data_set.dropna(subset = ["importance"])
data_set = data_set.reset_index(drop = True)
data_set['index'] = data_set.index
data_set.shape

(47241, 11)

In [8]:
data_set.head(1)

,index,no,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,0,0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


In [9]:
data_set.tail(1)

,index,no,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
47240,47240,34499,659762898834428032,연합뉴스,"""최신 갤럭시·아이폰, 4명 중 1명이 빌려쓴다""",SK텔레콤 홍보 모델들이 스마트폰 렌탈 서비스 'T렌탈'을 이용하고 있는 모습.\n...,SK텔레콤 홍보 모델들이 스마트폰 렌탈 서비스 'T렌탈'을 이용하고 있는 모습. S...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2018-06-17T09:00:00,-,0


In [10]:
# encoding the labels
possible_labels = data_set.importance.unique()
encoded_values = [1, 0, 2]

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = encoded_values[index]
label_dict

{'+': 1, '0': 0, '-': 2}

In [11]:
data_set['label'] = data_set.importance.replace(label_dict)

In [12]:
data_set.index.values

array([    0,     1,     2, ..., 47238, 47239, 47240], dtype=int64)

In [13]:
data_set.label.values

array([1, 0, 2, ..., 2, 2, 2], dtype=int64)

In [14]:
# train and validation split
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data_set.index.values,
                                                 data_set.label.values,
                                                 test_size = 0.2,
                                                 random_state = 42,
                                                 stratify = data_set.label.values)
data_set['data_type'] = ['not_set'] * data_set.shape[0]

In [15]:
data_set.loc[X_train, 'data_type'] = 'train'
data_set.loc[X_val, 'data_type'] = 'val'

data_set.groupby(['importance', 'label', 'data_type']).count()

index     no    uid  publisher  title  summary  \
importance label data_type                                                   
+          1     train      11725  11725  11725      11725  11725    11669   
                 val         2932   2932   2932       2932   2932     2922   
-          2     train      19088  19088  19088      19088  19088    18898   
                 val         4772   4772   4772       4772   4772     4733   
0          0     train       6979   6979   6979       6979   6979     6958   
                 val         1745   1745   1745       1745   1745     1743   

                            content  content_url  update_at  polarity  
importance label data_type                                             
+          1     train        11725        11725      11725     11724  
                 val           2932         2932       2932      2932  
-          2     train        19088        19088      19088     19082  
                 val           4772         4772       4772      4772  
0          0     train         6979         6979       6979      6974  
                 val           1745         1745       1745      1745

In [16]:
# tokenizer
from kobert_transformers import get_tokenizer
tokenizer = get_tokenizer()
tokenizer.tokenize('한국어 모델을 공유합니다')

['▁한국', '어', '▁모델', '을', '▁공유', '합니다']

In [15]:
data_set.head(1)

,index,no,uid,publisher,title,summary,content,content_url,update_at,importance,polarity,label,data_type
0,0,0,354652739944452416,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+,1,train


In [17]:
encoded_data_train = tokenizer.batch_encode_plus(
    data_set[data_set.data_type=='train'].content.values, 
    add_special_tokens = True, 
    return_attention_mask = True, 
    pad_to_max_length = True, 
    max_length = 512, 
    padding = True,
    truncation = True,
    return_tensors = 'pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    data_set[data_set.data_type=='val'].content.values, 
    add_special_tokens = True, 
    return_attention_mask = True, 
    pad_to_max_length = True, 
    padding = True,
    max_length = 512, 
    truncation = True,
    return_tensors = 'pt'
)

In [18]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data_set[data_set.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data_set[data_set.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [18]:
# torch.save(dataset_train, "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/2. processed data/dataset_train.pth")
# torch.save(dataset_val, "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/2. processed data/dataset_val.pth")

In [19]:
model = BertForSequenceClassification.from_pretrained("monologg/kobert",
                                                     num_labels = 3,
                                                     output_attentions = False,
                                                     output_hidden_states = False)
# model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [21]:
# data loaders
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler = RandomSampler(dataset_train), 
                              batch_size = batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler = SequentialSampler(dataset_val), 
                                   batch_size = batch_size)

In [20]:
# torch.save(dataloader_train, "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/2. processed data/dataloader_train.pth")
# torch.save(dataloader_validation, "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/2. processed data/dataloader_validation.pth")

In [22]:
# optimizer & scheduler
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [23]:
# performance metrics
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [40]:
# training loop
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [72]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [73]:
epochs = 5

for epoch in tqdm(range(1, epochs + 1)):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, 
                        desc = 'Epoch {:1d}'.format(epoch), 
                        leave = False, 
                        disable = False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    torch.save(model.state_dict(), f'C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\1. source code\\save_models\\20210809\\finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/296 [00:00<?, ?it/s]

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:79] data. DefaultCPUAllocator: not enough memory: you tried to allocate 201326592 bytes.

In [25]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [104]:
print(f1_score)

<function f1_score at 0x0000012A98C1B4C0>


In [26]:
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
#                                                       num_labels=len(label_dict),
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)
model = BertForSequenceClassification.from_pretrained("monologg/kobert",
                                                     num_labels = 3,
                                                     output_attentions = False,
                                                     output_hidden_states = False)
# model
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [28]:
os.path.join(model_path, file_name)

'C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/4. model result/kobert_20210810/models/finetuned_BERT_epoch_1.model'

In [42]:
model_path

'C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/4. model result/kobert_20210810/models/'

In [41]:
# import data
# data_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/1. input data"
# file_name = "merged_data_set_AIR.xlsx"
# data_set = pd.read_excel()

model_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/4. model result/kobert_20210810/models/"
file_name = "finetuned_BERT_epoch_1.model"
loaded_model = torch.load(os.path.join(model_path, file_name), map_location=torch.device('cpu'))
model.load_state_dict(loaded_model, strict = False)
_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 1325/1745

Class: +
Accuracy: 560/2932

Class: -
Accuracy: 526/4772



In [49]:
preds = torch.load('C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/4. model result/kobert_20210810/predicted/predicted_BERT_epoch_1.pth')
labels = torch.load('C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/4. model result/kobert_20210810/predicted/true_vals_BERT_epoch_1.pth')

label_dict_inverse = {v: k for k, v in label_dict.items()}
    
preds_flat = np.argmax(preds, axis=1).flatten()
labels_flat = labels.flatten()

In [114]:
from sklearn.metrics import confusion_matrix

save_path = 'C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/4. model result/kobert_20210810/predicted/'

for epoch in range(1, epochs + 1):
    preds = torch.load(save_path + f'predicted_BERT_epoch_{epoch}.pth')
    labels = torch.load(save_path + f'true_vals_BERT_epoch_{epoch}.pth')
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    cm = pd.DataFrame(confusion_matrix(labels_flat, preds_flat, labels = [1, 0, 2]))
    cm.loc['Pred_Total']= cm.sum()
    cm.loc[:,'Acutal_Total'] = cm.sum(axis=1)
    
    cm.columns = ['+', '0', '-', 'Actual_total']
    cm.index = ['+', '0', '-', 'Pred_total']
    
    
    pos_sensitivity = cm.iloc[0, 0] / cm.iloc[0, 3]
    neu_sensitivity = cm.iloc[1, 0] / cm.iloc[1, 3]
    neg_sensitivity = cm.iloc[2, 2] / cm.iloc[2, 3]
    
    
    pos_precision = cm.iloc[0, 0] / cm.iloc[3, 0]
    neu_precision = cm.iloc[1, 1] / cm.iloc[3, 1]
    neg_precision = cm.iloc[2, 2] / cm.iloc[3, 2]
    
    pos_fatal_error = cm.iloc[0, 2] / cm.iloc[0, 3]
    neg_fatal_error = cm.iloc[2, 0] / cm.iloc[2, 3]
    
    accuracy = (cm.iloc[0, 0] + cm.iloc[1, 1] + cm.iloc[2, 2]) / cm.iloc[3, 3]
    sensitivity = (pos_sensitivity + neu_sensitivity + neg_sensitivity) / 3
    precision = (pos_precision + neu_precision + neg_precision) / 3
    
    f1_score = 2 * ((precision * sensitivity) / (precision + sensitivity))
    
    
    print(f"[Epoch {epoch}]")
    print(cm)
    print("\n")
    print(f"정확도 = {accuracy}")
    print(f"f1 score = {f1_score}")
    print(f"총민감도 = {sensitivity}")
    print(f"총정밀도 = {precision}")
    
    print(f"긍정민감도 = {pos_sensitivity}")
    print(f"부정민감도 = {neg_sensitivity}")
    
    print(f"긍정정밀도 = {pos_precision}")
    print(f"부정정밀도 = {neg_precision}")
    
    print(f"긍정치명오분류율 = {pos_fatal_error}")
    print(f"부정치명오분류율 = {neg_fatal_error}")
    
    print("===================================================================================\n")

[Epoch 1]
               +    0     -  Actual_total
+           2267  111   554          2932
0            663  205   877          1745
-            508  150  4114          4772
Pred_total  3438  466  5545          9449


정확도 = 0.6970049740713303
f1 score = 0.6414391267808465
총민감도 = 0.6717491251503575
총정밀도 = 0.6137462755158521
긍정민감도 = 0.7731923601637107
부정민감도 = 0.8621123218776194
긍정정밀도 = 0.6593949970913322
부정정밀도 = 0.7419296663660956
긍정치명오분류율 = 0.18894952251023192
부정치명오분류율 = 0.10645431684828165

[Epoch 2]
               +    0     -  Actual_total
+           2347  133   452          2932
0            686  255   804          1745
-            555  173  4044          4772
Pred_total  3588  561  5300          9449


정확도 = 0.7033548523653297
f1 score = 0.6509005179964893
총민감도 = 0.6803480396289459
총정밀도 = 0.6238963943721942
긍정민감도 = 0.8004774897680764
부정민감도 = 0.8474434199497066
긍정정밀도 = 0.6541248606465998
부정정밀도 = 0.7630188679245283
긍정치명오분류율 = 0.15416098226466576
부정치명오분류율 = 0.11630343671416597



In [119]:
# for epoch in range(1, epochs + 1):
    
cm = pd.DataFrame(np.array([[2200, 1333, 448],
                          [471, 1464, 665],
                          [574, 2001, 4128]]))
cm.loc['Pred_Total']= cm.sum()
cm.loc[:,'Acutal_Total'] = cm.sum(axis=1)
    
cm.columns = ['+', '0', '-', 'Actual_total']
cm.index = ['+', '0', '-', 'Pred_total']
    
    
pos_sensitivity = cm.iloc[0, 0] / cm.iloc[0, 3]
neu_sensitivity = cm.iloc[1, 0] / cm.iloc[1, 3]
neg_sensitivity = cm.iloc[2, 2] / cm.iloc[2, 3]
    
    
pos_precision = cm.iloc[0, 0] / cm.iloc[3, 0]
neu_precision = cm.iloc[1, 1] / cm.iloc[3, 1]
neg_precision = cm.iloc[2, 2] / cm.iloc[3, 2]
    
pos_fatal_error = cm.iloc[0, 2] / cm.iloc[0, 3]
neg_fatal_error = cm.iloc[2, 0] / cm.iloc[2, 3]
    
accuracy = (cm.iloc[0, 0] + cm.iloc[1, 1] + cm.iloc[2, 2]) / cm.iloc[3, 3]
sensitivity = (pos_sensitivity + neu_sensitivity + neg_sensitivity) / 3
precision = (pos_precision + neu_precision + neg_precision) / 3
    
f1_score = 2 * ((precision * sensitivity) / (precision + sensitivity))
    
    
print(f"[딥서치]")
print(cm)
print("\n")
print(f"정확도 = {accuracy}")
print(f"f1 score = {f1_score}")
print(f"총민감도 = {sensitivity}")
print(f"총정밀도 = {precision}")
    
print(f"긍정민감도 = {pos_sensitivity}")
print(f"부정민감도 = {neg_sensitivity}")
    
print(f"긍정정밀도 = {pos_precision}")
print(f"부정정밀도 = {neg_precision}")
    
print(f"긍정치명오분류율 = {pos_fatal_error}")
print(f"부정치명오분류율 = {neg_fatal_error}")
    
print("===================================================================================\n")

[딥서치]
               +     0     -  Actual_total
+           2200  1333   448          3981
0            471  1464   665          2600
-            574  2001  4128          6703
Pred_total  3245  4798  5241         13284


정확도 = 0.5865703101475459
f1 score = 0.5105868734517719
총민감도 = 0.4498741556169255
총정밀도 = 0.5902430617800013
긍정민감도 = 0.5526249686008541
부정민감도 = 0.6158436520960764
긍정정밀도 = 0.6779661016949152
부정정밀도 = 0.7876359473382942
긍정치명오분류율 = 0.11253453906053755
부정치명오분류율 = 0.08563329852304938



In [120]:
# for epoch in range(1, epochs + 1):
    
cm = pd.DataFrame(np.array([[3008, 753, 235],
                          [799, 645, 774],
                          [404, 1059, 5583]]))
cm.loc['Pred_Total']= cm.sum()
cm.loc[:,'Acutal_Total'] = cm.sum(axis=1)
    
cm.columns = ['+', '0', '-', 'Actual_total']
cm.index = ['+', '0', '-', 'Pred_total']
    
    
pos_sensitivity = cm.iloc[0, 0] / cm.iloc[0, 3]
neu_sensitivity = cm.iloc[1, 0] / cm.iloc[1, 3]
neg_sensitivity = cm.iloc[2, 2] / cm.iloc[2, 3]
    
    
pos_precision = cm.iloc[0, 0] / cm.iloc[3, 0]
neu_precision = cm.iloc[1, 1] / cm.iloc[3, 1]
neg_precision = cm.iloc[2, 2] / cm.iloc[3, 2]
    
pos_fatal_error = cm.iloc[0, 2] / cm.iloc[0, 3]
neg_fatal_error = cm.iloc[2, 0] / cm.iloc[2, 3]
    
accuracy = (cm.iloc[0, 0] + cm.iloc[1, 1] + cm.iloc[2, 2]) / cm.iloc[3, 3]
sensitivity = (pos_sensitivity + neu_sensitivity + neg_sensitivity) / 3
precision = (pos_precision + neu_precision + neg_precision) / 3
    
f1_score = 2 * ((precision * sensitivity) / (precision + sensitivity))
    
    
print("[다음소프트]")
print(cm)
print("\n")
print(f"정확도 = {accuracy}")
print(f"f1 score = {f1_score}")
print(f"총민감도 = {sensitivity}")
print(f"총정밀도 = {precision}")
    
print(f"긍정민감도 = {pos_sensitivity}")
print(f"부정민감도 = {neg_sensitivity}")
    
print(f"긍정정밀도 = {pos_precision}")
print(f"부정정밀도 = {neg_precision}")
    
print(f"긍정치명오분류율 = {pos_fatal_error}")
print(f"부정치명오분류율 = {neg_fatal_error}")
    
print("===================================================================================\n")

[다음소프트]
               +     0     -  Actual_total
+           3008   753   235          3996
0            799   645   774          2218
-            404  1059  5583          7046
Pred_total  4211  2457  6592         13260


정확도 = 0.6965309200603318
f1 score = 0.621222918751525
총민감도 = 0.6351172201017534
총정밀도 = 0.6079235270558403
긍정민감도 = 0.7527527527527528
부정민감도 = 0.7923644621061595
긍정정밀도 = 0.7143196390406079
부정정밀도 = 0.8469356796116505
긍정치명오분류율 = 0.05880880880880881
부정치명오분류율 = 0.0573374964518876

